# Homework 6

## Problem 1: Markov Chains (40%)

Download the spreadsheet ENSO_to2020.xls. (And yes, the current status of ENSO, neutral, is for water year 2020.) [Not required for the homework, but if you’re curious to learn more about ENSO and its impacts, see this website: https://www.climate.gov/enso ]

Data:	
ENSO_to2020.xlsx

## A.

Using the time series of the phase of the El Niño Southern Oscillation (ENSO) (warm (El Nino) =1, neutral=2, cool (La Nina) =3) from 1900-2020, create a lag-1 Markov model of the ENSO phase.

In [3]:
import numpy as np
import scipy.stats as st
from scipy import sparse
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [4]:
df=pd.read_excel('ENSO_to2020.xlsx')

In [11]:
df_ENSO=df.iloc[8:,2:4]
df_ENSO.columns=['years','phase']
df_ENSO.head()

,years,phase
8,1900,1
9,1901,2
10,1902,2
11,1903,1
12,1904,3


In [15]:
df_ENSO['phase']=df_ENSO['phase'].astype(int)
data=np.array(df_ENSO['phase'])
data

array([1, 2, 2, 1, 3, 1, 1, 3, 2, 3, 3, 3, 1, 2, 1, 1, 2, 3, 3, 1, 1, 3,
       2, 3, 1, 3, 1, 2, 2, 2, 1, 1, 3, 2, 3, 2, 2, 2, 3, 3, 1, 1, 1, 3,
       3, 3, 2, 2, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 1, 2, 2, 2, 3, 1, 3,
       1, 2, 3, 1, 1, 3, 3, 1, 3, 3, 3, 1, 1, 2, 1, 2, 2, 1, 3, 3, 3, 1,
       1, 3, 2, 2, 1, 2, 2, 1, 3, 2, 1, 3, 3, 3, 2, 1, 2, 1, 3, 1, 3, 3,
       1, 3, 3, 2, 2, 1, 1, 3, 3, 1, 2])

In [16]:
# This counts the transitions from each state to the next and marks that count
S = sparse.csr_matrix((np.ones_like(data[:-1]), (data[:-1], data[1:])), dtype=np.float)

# This converts those counts to matrix form
tm = S.todense()
print(tm)

[[ 0.  0.  0.  0.]
 [ 0. 11. 12. 17.]
 [ 0. 12. 15.  9.]
 [ 0. 16. 10. 18.]]


We had 11 counts when the series transitioned from 1 to 1, 12 counts when it transitioned from 1 to 2, etc. We want to transition these from counts to frequencies. To do this, we need to normalize the transition matrix to get probabilities

In [17]:
tm_norm = np.zeros_like(tm)
for i in range(0,tm.shape[0]):
    tm_norm[i,:] = tm[i,:] / np.sum(tm[i,:])
    
print(tm_norm) # This is our normalized transition matrix.

[[       nan        nan        nan        nan]
 [0.         0.275      0.3        0.425     ]
 [0.         0.33333333 0.41666667 0.25      ]
 [0.         0.36363636 0.22727273 0.40909091]]


E:\tools\py\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# We take the above probabilities of transitions, and turn them into discrete CDF's.
# These will allow us to map random numbers generated from a uniform distribution into 
# transitions that follow these probability rules.
tm_cdf = np.cumsum(tm_norm,1)

## B.

Using this Markov model and a random number generator, simulate 5,000 years of ENSO data.

In [19]:
n_years = 5000
q = np.random.uniform(0,1,n_years); # uniformly distributed random numbers n_years long

initialstate = 2; # give it an initial state, doesn't really matter which

Nrand = np.zeros_like(q) # initialize an array of the proper size, with the initial state
Nrand[0] = initialstate;

# Now, just like we did when we created monte carlo simulations from empirical CDFs,
# we use our uniform random numbers to look up the next state in the transition matrix
for i in range(1,n_years):
    if q[i] <= tm_cdf[int(Nrand[i-1]),1]: #probability of transitioning from state i to 1
        Nrand[i] = 1;
    elif q[i] <= tm_cdf[int(Nrand[i-1]),2]: #transition from state i to 2
        Nrand[i] = 2;
    else:
        Nrand[i] = 3;

In [22]:
Nrand.shape

(5000,)

In [23]:
Nrand

array([2., 2., 1., ..., 1., 3., 3.])

Nrand is the simulated data for next 5000 years.

## C.

Using this stochastically generated data, answer the following questions. According to the model, what is the probability that three warm ENSO years would occur in a row? (Try refreshing the numbers several times to increase the sample size if the condition never happens.) What is the large-sample probability that three cool ENSO years would happen in a row?

In [26]:
# And how many times did state 3 appear 3 times?
Test3 = [Nrand[0:-2], Nrand[1:-1], Nrand[2:]] # stack our data 3 times, shifting it to the right by 1 each time
Test3 = np.stack(Test3, axis=1)

G2 = np.where((np.max(Test3, axis=1) == 3) & (np.min(Test3, axis=1) == 3))
# if both the maximum and the minimum are 3, then we have 3 threes in our sequence

frequencyof3threes = G2[0].size / Test3.shape[0]

print('Frequency of three warm periods in a row = {}%'.format(np.round(frequencyof3threes,3)))

Frequency of three warm periods in a row = 0.061%


In [29]:
# And how many times did state 3 appear 3 times?
Test3 = [Nrand[0:-2], Nrand[1:-1], Nrand[2:]] # stack our data 3 times, shifting it to the right by 1 each time
Test3 = np.stack(Test3, axis=1)

G2 = np.where((np.max(Test3, axis=1) == 1) & (np.min(Test3, axis=1) == 1))
frequencyof3threes = G2[0].size / Test3.shape[0]

print('Frequency of three cold periods in a row = {}%'.format(np.round(frequencyof3threes,3)))

Frequency of three cold periods in a row = 0.024%


## Problem 2: Application of Bayes Theorem (40%)

Following the Week 6 Lab, explore how the rating curve and its associated uncertainty change whether you use least squares fitting, direct monte carlo parameter estimation, or Bayesian MCMC fitting to determine the rating curve and 95% confidence intervals for the Lyell Fork streamflow site. Create plots and discuss what you did.

## Problem 3: Data Assimilation (20%)

Reading:	
reichle-2008.pdf

Read Reichle’s “Data assimilation methods in the Earth sciences”, available as reichle- 2008.pdf above and answer the following in your own words:

## A.

In the simple data assimilation system (section 2.1), what is the Kalman gain? What does it do, and why is it important?

## B.

According to Rolf Reichle, what are some of the greatest continuing challenges in data assimilation (see section 3)? List at least 3 and for each, give an example of how scientists are dealing with this challenge.